In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
from datetime import datetime, timedelta
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import src.config as config
from src.inference import get_feature_store
from src.data_utils import transform_ts_data_info_features
from src.inference import load_model_from_registry
from src.inference import get_model_predictions
from src.inference import get_feature_store


c:\Users\singh\Downloads\CDS500_Applied_ML_DS\Projects\CDA500P1\CDA500P1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

2025-03-04 21:21:10,737 INFO: Initializing external client
2025-03-04 21:21:10,737 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 21:21:11,691 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214689
Fetching data from 2025-02-04 02:21:10.735130+00:00 to 2025-03-05 01:21:10.735130+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (13.90s) 


,pickup_hour,pickup_location_id,rides
0,2025-02-04 03:00:00+00:00,2,0
1,2025-02-04 04:00:00+00:00,2,0
2,2025-02-04 05:00:00+00:00,2,0
3,2025-02-04 06:00:00+00:00,2,0
4,2025-02-04 07:00:00+00:00,2,0
...,...,...,...
174440,2025-03-04 21:00:00+00:00,263,76
174441,2025-03-04 22:00:00+00:00,263,58
174442,2025-03-04 23:00:00+00:00,263,30
174443,2025-03-05 00:00:00+00:00,263,22


In [3]:
model = load_model_from_registry()
predictions = get_model_predictions(model, features)
predictions["pickup_hour"] = current_date.ceil('h')
predictions

2025-03-04 21:21:36,410 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 21:21:36,417 INFO: Initializing external client
2025-03-04 21:21:36,418 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 21:21:37,050 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214689


,pickup_location_id,predicted_demand,pickup_hour
0,255,2.0,2025-03-05 03:00:00+00:00
1,209,4.0,2025-03-05 03:00:00+00:00
2,56,0.0,2025-03-05 03:00:00+00:00
3,171,0.0,2025-03-05 03:00:00+00:00
4,30,0.0,2025-03-05 03:00:00+00:00
...,...,...,...
246,166,10.0,2025-03-05 03:00:00+00:00
247,97,1.0,2025-03-05 03:00:00+00:00
248,65,1.0,2025-03-05 03:00:00+00:00
249,186,154.0,2025-03-05 03:00:00+00:00


In [4]:
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)
feature_group.insert(predictions, write_options={"wait_for_job": False})

2025-03-04 21:21:39,618 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 21:21:39,621 INFO: Initializing external client
2025-03-04 21:21:39,621 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 21:21:40,264 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214689


Uploading Dataframe: 100.00% |██████████| Rows 251/251 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214689/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)